In [2]:
import numpy as np
import glob
import csv
import os
import math

def demand_ideal_led_length(distance):
    ideal_led_length = (70/1000*3.16/1000)/(3.2/(1000*1080)*distance)
    return ideal_led_length

def demand_ideal_distance(distance,miss_value):
    ideal_distance_max = distance*(1+miss_value)
    ideal_distance_min = distance*(1-miss_value)
    return ideal_distance_max,ideal_distance_min


def get_width(top_right,top_left,bottom_right,bottom_left):
    #1st
    width1 = math.sqrt((int(top_right[0])-int(top_left[0]))**2+(int(top_right[1])-int(top_left[1]))**2)
    #2nd
    width2 = math.sqrt((int(bottom_right[0])-int(bottom_left[0]))**2+(int(bottom_right[1])-int(bottom_left[1]))**2)
    width = (width1 + width2)/2
    return width

def get_height(top_right,top_left,bottom_right,bottom_left):
    #1st
    height1 = math.sqrt((int(top_right[0])-int(bottom_right[0]))**2+(int(top_right[1])-int(bottom_right[1]))**2)
    #2nd
    height2 = math.sqrt((int(top_left[0])-int(bottom_left[0]))**2+(int(top_left[1])-int(bottom_left[1]))**2)
    height = (height1 + height2)/2
    return height

def get_coordinates(coord):
    x = int(coord[0])
    y = int(coord[1])
    coordinate = [x,y]
    return coordinate

色の精度検証

In [41]:
#全ての結果
import statistics

file = './result/color_check_2.csv'
with open(file, encoding='utf8', newline='') as f:
    csvreader = csv.DictReader(f)
    content = [row for row in csvreader]

color_lst = []
true_lst = []
false_lst = []
unknown_lst = []
color_cnt_lst = []
true_cnt = 0
false_cnt = 0
unknown_cnt = 0
real_r,real_g,real_b,real_w = 0,0,0,0
real_firstday_cnt = 0
real_thirdday_cnt = 0
real_shallow_cnt = 0
real_deep_cnt = 0
real_one_cnt = 0
real_one_half_cnt = 0
real_two_cnt = 0
firstday_cnt = 0
thirdday_cnt = 0
shallow_cnt = 0
deep_cnt = 0
one_cnt = 0
one_half_cnt = 0
two_cnt = 0

#ここ変える！！！！！！
for i in range(len(content)):
    file_name = content[i]['file_name']
    real_color = content[i]['real_color']
    estimated_color = content[i]['color']
    day = content[i]['day']
    depth = float(content[i]['depth'])
    distance = float(content[i]['distance'])
    
    if real_color == 'r':
        real_r += 1
    elif real_color == 'b':
        real_b += 1
    elif real_color == 'g':
        real_g += 1
    elif real_color == 'w':
        real_w += 1
    
    if estimated_color == '0':
        estimated_color = 'r'
    elif estimated_color == '1':
        estimated_color = 'g'
    elif estimated_color == '2':
        estimated_color = 'b'
    else:
        estimated_color = 'w'
    
    if real_color != 'n' and real_color != 'y':
        if day == '29':
                real_firstday_cnt+=1
        else:
            real_thirdday_cnt+=1
            
        if depth > 3:
            real_deep_cnt += 1
        else:
            real_shallow_cnt += 1
            
        if distance < 1:
            real_one_cnt += 1
        elif distance <2:
            real_one_half_cnt+=1
        else:
            real_two_cnt += 1
            
        if estimated_color == real_color:
            true_cnt +=1
            true_lst.append([file_name,day,depth,distance,estimated_color])
        else:
            false_cnt += 1
            false_lst.append([file_name,day,depth,distance,estimated_color])
    else:
        unknown_cnt += 1
        unknown_lst.append([file_name,day,depth,distance,estimated_color])

print(true_cnt, false_cnt,unknown_cnt)

color_cnt_lst.append([f'r:{real_r}',f'b:{real_b}',f'g:{real_g}',f'w:{real_w}'])
print(color_cnt_lst)

true_r,true_b,true_g,true_w =0,0,0,0

file = './result/color_true_lst.csv'
with open(file, encoding='utf8', newline='') as f:
    csvreader = csv.DictReader(f)
    content = [row for row in csvreader]
    
for i in range(len(content)):
    file_name = content[i]['file_name']
    color = content[i]['color']
    day = content[i]['day']
    depth = float(content[i]['depth'])
    distance = float(content[i]['real_distance_[m]'])
    
    if color == 'r':
        true_r += 1
    elif color == 'b':
        true_b += 1
    elif color == 'g':
        true_g += 1
    else:
        true_w += 1
        
    if day == '29':
        firstday_cnt+=1
    else:
        thirdday_cnt+=1
        
    if depth > 3:
        deep_cnt += 1
    else:
        shallow_cnt += 1
        
    if distance < 1:
        one_cnt += 1
    elif distance <2:
        one_half_cnt+=1
    else:
        two_cnt += 1
        
print(f'R:{(true_r/real_r)*100}')
print(f'B:{(true_b/real_b)*100}')
print(f'G:{(true_g/real_g)*100}')
print(f'W:{(true_w/real_w)*100}')

        

result_lst = []
for lst in [['r',true_r,real_r],['b',true_b,real_b],['g',true_g,real_g],['w',true_w,real_w],[]]:
    result_lst.append(lst)
for lst in [['29',firstday_cnt,real_firstday_cnt],['31',thirdday_cnt,real_thirdday_cnt],[]]:
    result_lst.append(lst)
for lst in [['deep',deep_cnt,real_deep_cnt],['shallow',shallow_cnt,real_shallow_cnt],[]]:
    result_lst.append(lst)
for lst in [['1m',one_cnt,real_one_cnt],['1.5m',one_half_cnt,real_one_half_cnt],['2m',two_cnt,real_two_cnt]]:
    result_lst.append(lst)

new_file = './result/color_analyze.csv'

# 新しいファイルに書き込む
with open(new_file, 'w', encoding='utf8', newline='') as f:
    csvwriter = csv.writer(f, delimiter=',')
    csvwriter.writerow(['condition','real_count','all_count'])
    csvwriter.writerows(result_lst)

# for lst in [true_lst,false_lst,unknown_lst]:                
#     #新しいファイル名を生成
#     if lst == true_lst:
#         new_file = f'./result/color_true_lst.csv'
#     elif lst == false_lst:
#         new_file = f'./result/color_false_lst.csv'
#     else:
#         new_file = f'./result/color_unknown_lst.csv'

#     # 新しいファイルに書き込む
#     with open(new_file, 'w', encoding='utf8', newline='') as f:
#         csvwriter = csv.writer(f, delimiter=',')
#         csvwriter.writerow(['file_name','day','depth','real_distance_[m]','color'])
#         csvwriter.writerows(lst)
            


196 59 42
[['r:68', 'b:92', 'g:49', 'w:46']]
R:73.52941176470588
B:94.56521739130434
G:61.224489795918366
W:63.04347826086957


結果の整理(異常値の除去完了後)

In [13]:
#全ての結果
import statistics

file = './result/allresult_new.csv'
with open(file, encoding='utf8', newline='') as f:
    csvreader = csv.DictReader(f)
    content = [row for row in csvreader]
    
bfo_real_distance = content[0]['real_distance_[m]']
print(bfo_real_distance)
distance_lst = []
average_lst = []

#ここ変える！！！！！！
for i in range(len(content)):
    # day = content[i]['day']
    # time = content[i]['time']
    color = content[i]['color']
    real_distance = content[i]['real_distance_[m]']
    distance_y = content[i]['distance_y_[m]']
    depth = content[i]['depth']
    csv_name = content[i]['file_name']
    
    if bfo_real_distance == real_distance:
        distance_lst.append(float(distance_y))
    elif bfo_real_distance == '4' and real_distance == '4.03':
        distance_lst.append(float(distance_y))
    else:
        # print(distance_lst)
        average = statistics.mean(distance_lst)
        stdev = statistics.stdev(distance_lst)
        print(f'{bfo_real_distance}:{average} and {stdev}')
        average_lst.append([bfo_real_distance,average,stdev])
        distance_lst = []
        bfo_real_distance = real_distance
        distance_lst.append(float(distance_y))

print(average_lst)
            
#新しいファイル名を生成
new_file = './result/estimated_distance_allresult_0218.txt'

# 新しいファイルに書き込む
with open(new_file, 'w', encoding='utf8', newline='') as f:
    csvwriter = csv.writer(f, delimiter='\t')
    csvwriter.writerow(['real_distance_[m]','distance_y_[m]','stdev_y'])
    csvwriter.writerows(average_lst)
            


0.97
0.97:0.8443017346938776 and 0.023337788048370105
1.5:1.2379133829787234 and 0.09588864534619519
2:1.719797368159204 and 0.04793489187568676
2.5:2.074277275510204 and 0.07972982387946903
3:2.5054973125 and 0.10591340386584072
3.5:2.9097918314606743 and 0.14977409273835135
4:3.377359496969697 and 0.20143910807434612
4.5:3.645106367346939 and 0.20205194726632117
5:4.374182787234043 and 0.325941547639326
[['0.97', 0.8443017346938776, 0.023337788048370105], ['1.5', 1.2379133829787234, 0.09588864534619519], ['2', 1.719797368159204, 0.04793489187568676], ['2.5', 2.074277275510204, 0.07972982387946903], ['3', 2.5054973125, 0.10591340386584072], ['3.5', 2.9097918314606743, 0.14977409273835135], ['4', 3.377359496969697, 0.20143910807434612], ['4.5', 3.645106367346939, 0.20205194726632117], ['5', 4.374182787234043, 0.325941547639326]]


In [38]:
#色別
import statistics

file = './result/allresult_new.csv'
with open(file, encoding='utf8', newline='') as f:
    csvreader = csv.DictReader(f)
    content = [row for row in csvreader]
    
bfo_real_distance = content[0]['real_distance_[m]']
print(bfo_real_distance)
distance_lst_R = []
distance_lst_G = []
distance_lst_B = []
distance_lst_W = []
average_lst = []
average_lst_R = []
average_lst_G = []
average_lst_B = []
average_lst_W = []

#ここ変える！！！！！！
for i in range(len(content)):
    color = content[i]['color']
    real_distance = content[i]['real_distance_[m]']
    distance_y = content[i]['distance_y_[m]']
    depth = content[i]['depth']
    csv_name = content[i]['file_name']
    
    if bfo_real_distance == real_distance:
        if color == '0':
            distance_lst_R.append(float(distance_y))
        elif color == '1':
            distance_lst_G.append(float(distance_y))
        elif color == '2':
            distance_lst_B.append(float(distance_y))
        else:
            distance_lst_W.append(float(distance_y))
        # distance_lst.append(float(distance_y))
    elif bfo_real_distance == '4' and real_distance == '4.03':
        if color == '0':
            distance_lst_R.append(float(distance_y))
        elif color == '1':
            distance_lst_G.append(float(distance_y))
        elif color == '2':
            distance_lst_B.append(float(distance_y))
        else:
            distance_lst_W.append(float(distance_y))
        # distance_lst.append(float(distance_y))
    else:
        print(f'{bfo_real_distance}')
        for distance_lst in [distance_lst_R, distance_lst_G,distance_lst_B,distance_lst_W]:
            # print(f'{len(distance_lst)}')
            if len(distance_lst) == 0:
                average_lst.append([bfo_real_distance,0,0])
            elif len(distance_lst) == 1:
                average_lst.append([bfo_real_distance,distance_lst[0],0])
            else:
                average = statistics.mean(distance_lst)
                stdev = statistics.stdev(distance_lst)
                # print(f'{bfo_real_distance}:{average} and {stdev}')
                average_lst.append([bfo_real_distance,average,stdev])
            
        distance_lst_R = []
        distance_lst_G = []
        distance_lst_B = []
        distance_lst_W = []
        
        bfo_real_distance = real_distance
        if color == '0':
            distance_lst_R.append(float(distance_y))
        elif color == '1':
            distance_lst_G.append(float(distance_y))
        elif color == '2':
            distance_lst_B.append(float(distance_y))
        else:
            distance_lst_W.append(float(distance_y))

# print(average_lst)

for p in range(int(len(average_lst)/4)):
    average_lst_R.append(average_lst[0+4*p])
    average_lst_G.append(average_lst[1+4*p])
    average_lst_B.append(average_lst[2+4*p])
    average_lst_W.append(average_lst[3+4*p])

for list in [average_lst_R,average_lst_G,average_lst_B,average_lst_W]:
    print(list)            
    #新しいファイル名を生成
    if list == average_lst_R:
        new_file = './result/estimated_distance_Red.txt'
    elif list == average_lst_G:
        new_file = './result/estimated_distance_Green.txt'
    elif list == average_lst_B:
        new_file = './result/estimated_distance_Blue.txt'
    else:
        new_file = './result/estimated_distance_White.txt'

    # 新しいファイルに書き込む
    with open(new_file, 'w', encoding='utf8', newline='') as f:
        csvwriter = csv.writer(f, delimiter='\t')
        csvwriter.writerow(['real_distance_[m]','distance_y_[m]','stdev_y'])
        csvwriter.writerows(list)
            

0.97
0.97
11
7
18
13
1.5
12
13
15
7
2
38
53
68
42
2.5
15
36
31
16
3
24
17
41
14
3.5
24
25
40
0
4
38
69
44
14
4.5
9
27
13
0
5
1
45
1
0
[['0.97', 0.8576799090909091, 0.027693656852985474], ['1.5', 1.2192379166666667, 0.019667707337194036], ['2', 1.7076282105263159, 0.040588184900560015], ['2.5', 2.0299138, 0.060833721151770964], ['3', 2.4390839166666667, 0.10233185133243311], ['3.5', 2.840759875, 0.11113583000381053], ['4', 3.277624552631579, 0.14547068106735409], ['4.5', 3.472724, 0.14925178327912864], ['5', 4.035405, 0]]
[['0.97', 0.8320857142857143, 0.034326936316903714], ['1.5', 1.2690170769230769, 0.1810651220378373], ['2', 1.7086083773584906, 0.03310258846329463], ['2.5', 2.091453583333333, 0.07025979608541638], ['3', 2.530587823529412, 0.08100515276607043], ['3.5', 2.94643236, 0.0949085283509338], ['4', 3.4112607971014492, 0.21790183977529476], ['4.5', 3.625284777777778, 0.16774629458347856], ['5', 4.382811955555556, 0.32912998796862203]]
[['0.97', 0.844396, 0.01673638759252004], 

In [46]:
#深さ別
import statistics

file = './result/allresult_new.csv'
with open(file, encoding='utf8', newline='') as f:
    csvreader = csv.DictReader(f)
    content = [row for row in csvreader]
    
bfo_real_distance = content[0]['real_distance_[m]']
print(bfo_real_distance)
distance_lst_shallow = []
distance_lst_deep = []
average_lst = []
average_shallow = []
average_deep = []
shallow_length = 0
deep_length = 0

#ここ変える！！！！！！
for i in range(len(content)):
    color = content[i]['color']
    real_distance = content[i]['real_distance_[m]']
    distance_y = content[i]['distance_y_[m]']
    depth = float(content[i]['depth'])
    csv_name = content[i]['file_name']

    if bfo_real_distance == real_distance:
        if depth <=3:
            distance_lst_shallow.append(float(distance_y))
        else:
            distance_lst_deep.append(float(distance_y))
    elif bfo_real_distance == '4' and real_distance == '4.03':
        if depth <=3:
            distance_lst_shallow.append(float(distance_y))
        else:
            distance_lst_deep.append(float(distance_y))
    else:
        for distance_lst in [distance_lst_shallow,distance_lst_deep]:
            if distance_lst == distance_lst_shallow:
                shallow_length += len(distance_lst)
            else:
                deep_length += len(distance_lst)
            average = statistics.mean(distance_lst)
            stdev = statistics.stdev(distance_lst)
            # print(f'{bfo_real_distance}:{average} and {stdev}')
            average_lst.append([bfo_real_distance,average,stdev])
        distance_lst_shallow = []
        distance_lst_deep = []
        
        bfo_real_distance = real_distance
        if depth <=3:
            distance_lst_shallow.append(float(distance_y))
        else:
            distance_lst_deep.append(float(distance_y))

print(shallow_length,deep_length)
print(average_lst)


for p in range(int(len(average_lst)/2)):
    average_shallow.append(average_lst[0+2*p])
    average_deep.append(average_lst[1+2*p])

for list in [average_shallow,average_deep]:
    #新しいファイル名を生成
    if list == average_shallow:
        new_file = './result/estimated_distance_shallow.txt'
    else:
        new_file = './result/estimated_distance_deep.txt'

    # 新しいファイルに書き込む
    with open(new_file, 'w', encoding='utf8', newline='') as f:
        csvwriter = csv.writer(f, delimiter='\t')
        csvwriter.writerow(['real_distance_[m]','distance_y_[m]','stdev_y'])
        csvwriter.writerows(list)


0.97
417 424
[['0.97', 0.8615325217391304, 0.01285332866285676], ['0.97', 0.8290591153846154, 0.019635948195749396], ['1.5', 1.2420224583333332, 0.13445367628358884], ['1.5', 1.233625652173913, 0.016970317599236992], ['2', 1.7228962935779817, 0.04643792481691988], ['2', 1.7161258152173913, 0.049654176278200186], ['2.5', 2.0720112037037035, 0.06378503761688877], ['2.5', 2.0770583636363638, 0.09649246176463516], ['3', 2.531000387755102, 0.10201963150026902], ['3', 2.478909, 0.10436103487225822], ['3.5', 2.948235132075472, 0.09114094131323743], ['3.5', 2.85319475, 0.1962832749503162], ['4', 3.4650929705882354, 0.20255860295763392], ['4', 3.315855618556701, 0.1772299706607352], ['4.5', 3.7795674166666666, 0.16143451072208723], ['4.5', 3.51602376, 0.14573400474159534], ['5', 4.2328266153846155, 0.11143769368316565], ['5', 4.428230735294117, 0.36426939001166886]]


結果の整理(異常値の排除) 

In [ ]:
#異常値となったデータについての考察  
json_lst = []
error_led_lst = []

file = './result/allresult_error_new.csv'
with open(file, encoding='utf8', newline='') as f:
    csvreader = csv.DictReader(f)
    content = [row for row in csvreader]

#ここ変える！！！！！！
for i in range(len(content)):
    differences = []
    widths = []
    heights = []
    json_lst = []
    coordinates = []
    day = content[i]['day']
    time = content[i]['time']
    csv_name = content[i]['file_name']
    json_name = content[i]['json']
    error_width = content[i]['led_width_[pixel]']
    error_height = content[i]['led_height_[pixel]']
    print(error_width,error_height)
    
    if not json_name == '':
        json_path = f'./result/output_202401{day}/202401{day}/{time}/{json_name}.json'
        csv_path = f'./result/output_202401{day}/{csv_name}'
        
    with open(csv_path, encoding='utf8', newline='') as k:
        csvreader_csv = csv.DictReader(k)
        content_csv = [row for row in csvreader_csv]   
    
    with open(json_path, encoding='utf8', newline='') as p:
        csvreader_json = csv.reader(p)
        content_json = [row for row in csvreader_json]
        led_num = int(len(content_json)/36)
        for z in range(led_num):
            # print(path)
            top_right = [content_json[20+36*z][0],content_json[21+36*z][0]]
            bottom_right = [content_json[24+36*z][0],content_json[25+36*z][0]]
            bottom_left = [content_json[28+36*z][0],content_json[29+36*z][0]]
            top_left = [content_json[32+36*z][0],content_json[33+36*z][0]]
            width = get_width(top_right,top_left,bottom_right,bottom_left)
            widths.append(width)
            height = get_height(top_right,top_left,bottom_right,bottom_left)
            heights.append(height)
            for coord in [top_right,bottom_right,bottom_left,top_left]:
                coord = get_coordinates(coord)
                coordinates.append(coord)
            json_lst.append([csv_name,json_name,width,height,coordinates[0],coordinates[1],coordinates[2],coordinates[3]])
            # print(json_lst)
            coordinates = []
            
        for x in range(led_num):
            # print(x)
            # print(widths[x])
            difference_width = abs(float(error_width) - float(widths[x]))
            difference_height = abs(float(error_height) - float(heights[x]))
            difference = difference_width + difference_height
            differences.append(difference)
        sorted_differences = sorted(differences)
        # print(sorted_differences)
        min_value = sorted_differences[0]
        min_value_index = differences.index(min_value)
        # print(json_lst[min_value_index])
        error_led_lst.append(json_lst[min_value_index])
        # print(error_led_lst)
                
                
            
#新しいファイル名を生成
new_file = './result/error_find_list.csv'
error_file = './result/allresult_error.csv'

# 新しいファイルに書き込む
with open(new_file, 'w', encoding='utf8', newline='') as f:
    csvwriter = csv.writer(f, delimiter=',')
    csvwriter.writerow(['file_name','json','led_width_[pixel]','led_height_[pixel]','top_right','bottom_right','bottom_left','top_left'])
    csvwriter.writerows(error_led_lst)
            


In [32]:
#異常値となったデータについての考察  
error_lst = []

file = './result/allresult_error_pre.csv'
with open(file, encoding='utf8', newline='') as f:
    csvreader = csv.DictReader(f)
    content = [row for row in csvreader]

#ここ変える！！！！！！
for i in range(len(content)):
    csv_name = content[i]['file_name']
    error_lst.append([csv_name])
            
#新しいファイル名を生成
new_file = './result/final_error_list.csv'

# 新しいファイルに書き込む
with open(new_file, 'w', encoding='utf8', newline='') as f:
    csvwriter = csv.writer(f, delimiter=',')
    csvwriter.writerow(['file_name'])
    csvwriter.writerows(error_lst)
            

In [ ]:
#検出したLEDの描画
import os
import json
import cv2
import matplotlib.pyplot as plt
import csv

def transform_coordinates(coord, width_org, height_org, width_new, height_new):
    x_ratio = int(coord[1:4]) / width_org
    if coord[8] == ']':
        y_ratio = int(coord[6:8]) / height_org
    else:
        y_ratio = int(coord[6:9]) / height_org

    x_min_new = int(x_ratio * width_new)
    y_min_new = int(y_ratio * height_new)

    return [x_min_new, y_min_new]

    
def draw_box(csv_path):
    with open(csv_path, encoding='utf8', newline='') as k:
        csvreader_csv = csv.DictReader(k)
        content_csv = [row for row in csvreader_csv]
    
    bfo_image_name = '0.97m_dep1m_0129_114602'
    bfo_image_name_num = 1
    
    for i in range(len(content_csv)):
        top_right,bottom_right,bottom_left,top_left = content_csv[i]['top_right'],content_csv[i]['bottom_right'],content_csv[i]['bottom_left'],content_csv[i]['top_left']
        top_right = transform_coordinates(top_right,1920, 1080, 2304, 1296)
        bottom_right = transform_coordinates(bottom_right,1920, 1080, 2304, 1296)
        bottom_left = transform_coordinates(bottom_left,1920, 1080, 2304, 1296)
        top_left =transform_coordinates(top_left,1920, 1080, 2304, 1296)
        csv_name = content_csv[i]['file_name']
        day = content_csv[i]['day']
        image_name = csv_name.rsplit('.', 1)[0]
        image_path = f'./result/output_202401{day}/{image_name}.jpg'
        if not os.path.exists(image_path):
            # パスを分割して、数字部分を取得
            print('パスがなかったので新しいパスを作ります')
            path_parts = image_path.split('_')
            last_part = path_parts[-1]
            num_str = last_part.split('.')[0]  # 拡張子を取り除く

            # 数字を+1して新しい数字部分を生成
            new_num = str(int(num_str) + 1).zfill(len(num_str))
            # 新しいパスを生成
            image_path = '_'.join(path_parts[:-1]) + '_' + new_num + '.jpg'
            print("新しいパス:", image_path)
        else:
            print("指定されたパスは存在します。")
        
        print(image_path)
        image = cv2.imread(image_path)
        cv2.line(image, top_right, bottom_right, (255,0,255), 3, cv2.LINE_AA)
        cv2.line(image, bottom_right,bottom_left, (255,0,255), 3, cv2.LINE_AA)
        cv2.line(image, bottom_left,top_left, (255,0,255), 3, cv2.LINE_AA)
        cv2.line(image, top_right, top_left, (255,0,255), 3, cv2.LINE_AA)
        
        if image_name == bfo_image_name:
            for j in range(bfo_image_name_num):
                save_path = f'./result/error_image/annotated_{image_name}_{bfo_image_name_num}.jpg'
                bfo_image_name_num += 1
                cv2.imwrite(save_path, image)
                print(f"保存された画像: {save_path}")
        else:
            save_path = f'./result/error_image/annotated_{image_name}.jpg'
            cv2.imwrite(save_path, image)
            print(f"保存された画像: {save_path}")
            bfo_image_name_num = 1
        
        bfo_image_name = image_name
        
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        plt.imshow(image_rgb)
        plt.axis('off')
        plt.show()

csv_path = './result/error_find_list.csv'      
draw_box(csv_path)


In [35]:
#全ての結果☑(LEDが複数個検出されたときに区別)

import glob
import csv
import os

new_list = []
error_lst = []

for day in ['29','30','31']:
    # files = glob.glob("./result/test/0.97m_dep9m_0129_??????.csv")
    files = glob.glob(f"./result/output_202401{day}/*.csv")


#全ての結果を保存する(距離と推定距離)        
    for file in files:
        with open(file, encoding='utf8', newline='') as f:
            csvreader = csv.DictReader(f)
            content = [row for row in csvreader]

#ここ変える！！！！！！
            for i in range(len(content)):
                distance = float(content[i]['real_distance_[m]'])
                ideal_distance_max,ideal_distance_min = demand_ideal_distance(distance,0.25)
                file_name = os.path.basename(file)
                if float(content[i]['distance_y_[m]']) <= ideal_distance_min or float(content[i]['distance_y_[m]']) >= ideal_distance_max:
                    # error_lst.append([file_name,float(content[i]['real_distance_[m]']),float(content[i]['depth_[m]']),
                    #                 float(content[i]['led_width_[pixel]']),float(content[i]['led_height_[pixel]']),
                    #                 float(content[i]['distance_x_[m]']),float(content[i]['distance_y_[m]']),float(content[i]['category'])])
                    new_list.append([file_name,float(content[i]['real_distance_[m]']),float(content[i]['depth_[m]']),
                                    float(content[i]['distance_x_[m]']),float(content[i]['distance_y_[m]']),float(content[i]['category']),0])
                else:
                    new_list.append([file_name,float(content[i]['real_distance_[m]']),float(content[i]['depth_[m]']),
                                    float(content[i]['distance_x_[m]']),float(content[i]['distance_y_[m]']),float(content[i]['category']),1])
                
                
print(len(new_list))
# print(len(error_lst))                
# 新しいファイル名を生成
new_file = './result/allresult.csv'
error_file = './result/allresult_error.csv'

# 新しいファイルに書き込む
with open(new_file, 'w', encoding='utf8', newline='') as f:
    csvwriter = csv.writer(f, delimiter=',')
    csvwriter.writerow(['file_name','real_distance_[m]','depth','distance_x_[m]','distance_y_[m]','color','error OK:0'])
    csvwriter.writerows(new_list)

# with open(error_file, 'w', encoding='utf8', newline='') as f:
#     csvwriter = csv.writer(f, delimiter=',')
#     csvwriter.writerow(['file_name','real_distance_[m]','depth','led_width_[pixel]','led_height_[pixel]','distance_x_[m]','distance_y_[m]','color'])
#     csvwriter.writerows(error_lst)


921


In [23]:
#全ての結果☑(LEDが複数個検出されたときに区別)

import glob
import csv
import os

new_list = []
error_lst = []

for day in ['29','30','31']:
    # files = glob.glob("./result/test/0.97m_dep9m_0129_??????.csv")
    files = glob.glob(f"./result/output_202401{day}/*.csv")


#全ての結果を保存する(距離と推定距離)        
    for file in files:
        with open(file, encoding='utf8', newline='') as f:
            csvreader = csv.DictReader(f)
            content = [row for row in csvreader]

#ここ変える！！！！！！
            for i in range(len(content)):
                distance = float(content[i]['real_distance_[m]'])
                ideal_distance_max,ideal_distance_min = demand_ideal_distance(distance,0.25)
                differences = []
                
                if len(content) > 1:
                    if i == 0:
                        pick_best_lst = []
                    pick_best_lst.append([float(content[i]['real_distance_[m]']),float(content[i]['distance_x_[m]']),float(content[i]['distance_y_[m]'])])
                    # print(pick_best_lst)
                    if i == len(content)-1:
                        for p in range(len(content)):
                            difference = abs(float(content[p]['distance_y_[m]']) - float(content[p]['real_distance_[m]']))
                            differences.append(difference)
                        # print(differences)
                        sorted_differences = sorted(differences)
                        # print(sorted_differences)
                        min_value = sorted_differences[0]
                        min_value_index = differences.index(min_value)
                        # print(min_value_index)
                        second_min_value = sorted_differences[1]
                        second_min_value_index = differences.index(second_min_value)
                        # print(second_min_value_index)
                        new_list.append(pick_best_lst[min_value_index])
                        if float(pick_best_lst[min_value_index][2]) <= ideal_distance_min or float(pick_best_lst[second_min_value_index][2]) >= ideal_distance_max:
                            error_lst.append([float(content[min_value_index]['real_distance_[m]']),float(content[min_value_index]['distance_x_[m]']),float(content[min_value_index]['distance_y_[m]'])])
                            print(f'Error!:{file}')
                            print(min_value_index)
                            print(float(content[min_value_index]['distance_y_[m]']))
                        
                        if float(pick_best_lst[second_min_value_index][2]) >= ideal_distance_min and float(pick_best_lst[second_min_value_index][2]) <= ideal_distance_max:
                            new_list.append(pick_best_lst[second_min_value_index])
                        else:
                            break
                    else:
                        continue
                else:
                    new_list.append([float(content[i]['real_distance_[m]']),float(content[i]['distance_x_[m]']),float(content[i]['distance_y_[m]'])])
                
                
print(len(new_list))                
# 新しいファイル名を生成
new_file = './result/allresult.csv'

# 新しいファイルに書き込む
with open(new_file, 'w', encoding='utf8', newline='') as f:
    csvwriter = csv.writer(f, delimiter=',')
    csvwriter.writerow(['real_distance_[m]','distance_x_[m]','distance_y_[m]'])
    csvwriter.writerows(new_list)


Error!:./result/output_20240129\0.97m_dep1m_0129_114602.csv
0
0.8535484582542666
Error!:./result/output_20240129\0.97m_dep9m_0129_121649.csv
0
0.8119486240912142
Error!:./result/output_20240129\0.97m_dep9m_0129_121736.csv
0
0.7989546595998123
Error!:./result/output_20240129\0.97m_dep9m_0129_121829.csv
2
0.8534932667415126
Error!:./result/output_20240129\0.97m_dep9m_0129_121922.csv
1
0.8027474299376205
Error!:./result/output_20240129\0.97m_dep9m_0129_122009.csv
1
0.8504139978636422
Error!:./result/output_20240129\0.97m_dep9m_0129_122050.csv
3
0.838225014170619
Error!:./result/output_20240129\0.97m_dep9m_0129_122632.csv
0
0.8399131630610501
Error!:./result/output_20240129\0.97m_dep9m_0129_122716.csv
1
0.8444032213290293
Error!:./result/output_20240129\0.97m_dep9m_0129_122853.csv
0
1.8410894704798157
Error!:./result/output_20240129\0.97m_dep9m_0129_123032.csv
2
0.8508437889533156
Error!:./result/output_20240129\0.97m_dep9m_0129_123046.csv
2
0.8496560604018795
Error!:./result/output_202401

In [16]:
#全ての結果☑

import glob
import csv
import os

new_list = []

for day in ['29','30','31']:
    # files = glob.glob("./result/test/0.97m_dep9m_0129_??????.csv")
    files = glob.glob(f"./result/output_202401{day}/*.csv")


#全ての結果を保存する(距離と推定距離)        
    for file in files:
        with open(file, encoding='utf8', newline='') as f:
            csvreader = csv.DictReader(f)
            content = [row for row in csvreader]

#ここ変える！！！！！！
            for i in range(len(content)):
                distance = float(content[i]['real_distance_[m]'])
                ideal_distance_max,ideal_distance_min = demand_ideal_distance(distance,0.25)
                if (float(content[i]['distance_x_[m]']) >= ideal_distance_min and float(content[i]['distance_x_[m]']) <= ideal_distance_max) or (float(content[i]['distance_y_[m]']) >= ideal_distance_min and float(content[i]['distance_y_[m]']) <= ideal_distance_max):
                    new_list.append([float(content[i]['real_distance_[m]']),float(content[i]['distance_x_[m]']),float(content[i]['distance_y_[m]'])])
                else:
                    continue
                
                
print(len(new_list))                
# 新しいファイル名を生成
new_file = './result/allresult.csv'

# 新しいファイルに書き込む
with open(new_file, 'w', encoding='utf8', newline='') as f:
    csvwriter = csv.writer(f, delimiter=',')
    csvwriter.writerow(['real_distance_[m]','distance_x_[m]','distance_y_[m]'])
    csvwriter.writerows(new_list)


846


In [18]:
#深さ別☑

import glob
import csv
import os

depth_shallow = ['1','1.0','1.3','1.4','1.5','1.6','1.7','1.9','2']
depth_deep = ['5.1','5.7','5.9','6.0','6','6.1','6.2','6.3','6.6','7.0','7','9']

#浅いとき
new_list = []
for day in ['29','30','31']:
    for depth in depth_shallow:
        files_shallow = glob.glob(f"./result/output_202401{day}/*m_dep{depth}m_01{day}_??????.csv")

        #全ての結果を保存する(距離と推定距離)        
        for file in files_shallow:
            with open(file, encoding='utf8', newline='') as f:
                csvreader = csv.DictReader(f)
                content = [row for row in csvreader]

                for i in range(len(content)):
                    distance = float(content[i]['real_distance_[m]'])
                    ideal_distance_max,ideal_distance_min = demand_ideal_distance(distance,0.25)
                    if (float(content[i]['distance_x_[m]']) >= ideal_distance_min and float(content[i]['distance_x_[m]']) <= ideal_distance_max) or (float(content[i]['distance_y_[m]']) >= ideal_distance_min and float(content[i]['distance_y_[m]']) <= ideal_distance_max):
                        new_list.append([float(content[i]['real_distance_[m]']),float(content[i]['distance_x_[m]']),float(content[i]['distance_y_[m]'])])
                    else:
                        continue
                    
print(len(new_list))
                                        
# 新しいファイル名を生成
new_file = './result/shallow_result.csv'

# 新しいファイルに書き込む
with open(new_file, 'w', encoding='utf8', newline='') as f:
    csvwriter = csv.writer(f, delimiter=',')
    csvwriter.writerow(['real_distance_[m]','distance_x_[m]','distance_y_[m]'])
    csvwriter.writerows(new_list)
    

#深いとき
new_list = []
for day in ['29','30','31']:
    for depth in depth_deep:
        files_deep = glob.glob(f"./result/output_202401{day}/*m_dep{depth}m_01{day}_??????.csv")
        # print(files_deep)
        # print(len(files_deep))

        #全ての結果を保存する(距離と推定距離)        
        for file in files_deep:
            with open(file, encoding='utf8', newline='') as f:
                csvreader = csv.DictReader(f)
                content = [row for row in csvreader]

                for i in range(len(content)):
                    distance = float(content[i]['real_distance_[m]'])
                    ideal_distance_max,ideal_distance_min = demand_ideal_distance(distance,0.25)
                    if (float(content[i]['distance_x_[m]']) >= ideal_distance_min and float(content[i]['distance_x_[m]']) <= ideal_distance_max) or (float(content[i]['distance_y_[m]']) >= ideal_distance_min and float(content[i]['distance_y_[m]']) <= ideal_distance_max):
                        new_list.append([float(content[i]['real_distance_[m]']),float(content[i]['distance_x_[m]']),float(content[i]['distance_y_[m]'])])
                    else:
                        continue
print(len(new_list))
                                        
# 新しいファイル名を生成
new_file = './result/deep_result.csv'

# 新しいファイルに書き込む
with open(new_file, 'w', encoding='utf8', newline='') as f:
    csvwriter = csv.writer(f, delimiter=',')
    csvwriter.writerow(['real_distance_[m]','distance_x_[m]','distance_y_[m]'])
    csvwriter.writerows(new_list)



419
427


In [19]:
#日にち別☑

import glob
import csv
import os

for day in ['29','30','31']:
    new_list = []
    files = glob.glob(f"./result/output_202401{day}/*.csv")


#全ての結果を保存する(距離と推定距離)        
    for file in files:
        with open(file, encoding='utf8', newline='') as f:
            csvreader = csv.DictReader(f)
            content = [row for row in csvreader]

            for i in range(len(content)):
                distance = float(content[i]['real_distance_[m]'])
                ideal_distance_max,ideal_distance_min = demand_ideal_distance(distance,0.25)
                if (float(content[i]['distance_x_[m]']) >= ideal_distance_min and float(content[i]['distance_x_[m]']) <= ideal_distance_max) or (float(content[i]['distance_y_[m]']) >= ideal_distance_min and float(content[i]['distance_y_[m]']) <= ideal_distance_max):
                    new_list.append([float(content[i]['real_distance_[m]']),float(content[i]['distance_x_[m]']),float(content[i]['distance_y_[m]'])])
                else:
                    continue

    print(len(new_list))

    # 新しいファイル名を生成
    new_file = f'./result/01{day}_result.csv'

    # 新しいファイルに書き込む
    with open(new_file, 'w', encoding='utf8', newline='') as f:
        csvwriter = csv.writer(f, delimiter=',')
        csvwriter.writerow(['real_distance_[m]','distance_x_[m]','distance_y_[m]'])
        csvwriter.writerows(new_list)



210
418
218


In [23]:
#色別

import glob
import csv
import os

colors = ['Red','Blue','Green','White']

R_list = []
B_list = []
G_list = []
W_list = []


for day in ['29','30','31']:
    # files = glob.glob("./result/test/0.97m_dep9m_0129_??????.csv")
    files = glob.glob(f"./result/output_202401{day}/*.csv")


#全ての結果を保存する(距離と推定距離)        
    for file in files:
        with open(file, encoding='utf8', newline='') as f:
            csvreader = csv.DictReader(f)
            content = [row for row in csvreader]

#ここ変える！！！！！！
            for i in range(len(content)):
                distance = float(content[i]['real_distance_[m]'])
                ideal_distance_max,ideal_distance_min = demand_ideal_distance(distance,0.25)
                if (float(content[i]['distance_x_[m]']) >= ideal_distance_min and float(content[i]['distance_x_[m]']) <= ideal_distance_max) or (float(content[i]['distance_y_[m]']) >= ideal_distance_min and float(content[i]['distance_y_[m]']) <= ideal_distance_max):
                    if content[i]['category'] == '0':
                        R_list.append([float(content[i]['real_distance_[m]']),float(content[i]['distance_x_[m]']),float(content[i]['distance_y_[m]'])])
                    elif content[i]['category'] == '1':
                        G_list.append([float(content[i]['real_distance_[m]']),float(content[i]['distance_x_[m]']),float(content[i]['distance_y_[m]'])])
                    elif content[i]['category'] == '2':
                        B_list.append([float(content[i]['real_distance_[m]']),float(content[i]['distance_x_[m]']),float(content[i]['distance_y_[m]'])]) 
                    else:
                        W_list.append([float(content[i]['real_distance_[m]']),float(content[i]['distance_x_[m]']),float(content[i]['distance_y_[m]'])]) 
                else:
                    continue

print(len(R_list),len(G_list),len(B_list),len(W_list))

# 新しいファイル名を生成
for color in colors:
    new_file = f'./result/{color}_result.csv'

    # 新しいファイルに書き込む
    with open(new_file, 'w', encoding='utf8', newline='') as f:
        csvwriter = csv.writer(f, delimiter=',')
        csvwriter.writerow(['real_distance_[m]','distance_x_[m]','distance_y_[m]'])
        if color == 'Red':
            csvwriter.writerows(R_list)
        elif color == 'Green':
            csvwriter.writerows(G_list)
        elif color == 'Blue':
            csvwriter.writerows(B_list)
        else:
            csvwriter.writerows(W_list)


173 294 273 106


水中モード→空気中モード

In [30]:
import glob
import csv
import os

# files = glob.glob("./result/test/0.97m_dep9m_0129_??????.csv")
files = glob.glob("./result/test/*.csv")

new_file_dir = "./result/test/update" 
if not os.path.exists(new_file_dir):
    os.mkdir(new_file_dir)
    
for file in files:
    new_values_x = []
    new_values_y = []

    with open(file, encoding='utf8', newline='') as f:
        csvreader = csv.DictReader(f)
        content = [row for row in csvreader]

        for i in range(len(content)):
            # print("Original value:", content[i]['distance_x_[m]'])
            value_x = float(content[i]['distance_x_[m]'])
            new_value_x = value_x / 1.33
            new_values_x.append(new_value_x)
            content[i]['distance_x_[m]'] = str(new_value_x)  # 新しい値で置き換え
            
            value_y = float(content[i]['distance_y_[m]'])
            new_value_y = value_y / 1.33
            new_values_y.append(new_value_y)
            content[i]['distance_y_[m]'] = str(new_value_y)  # 新しい値で置き換え
            # print("New value:", content[i]['distance_x_[m]'])
            
    if len(content)>0:
        # 新しいファイル名を生成
        file_name = os.path.basename(file)
        # new_file = f'{new_file_dir}/' + os.path.splitext(file_name)[0] + '_new.csv'
        new_file = f'{new_file_dir}/' + os.path.splitext(file_name)[0] + '.csv'

        # 新しいファイルに書き込む
        with open(new_file, 'w', encoding='utf8', newline='') as f:
            csvwriter = csv.DictWriter(f, fieldnames=content[0].keys(), delimiter=',')
            csvwriter.writeheader()
            csvwriter.writerows(content)
    else:
        continue


In [4]:
import glob
import csv

files = glob.glob("./result/test/0.97m_dep9m_0129_??????.csv")

for file in files:
    new_values = []

    with open(file, encoding='utf8', newline='') as f:
        csvreader = csv.DictReader(f)
        content = [row for row in csvreader]

        for i in range(len(content)):
            print("Original value:", content[i]['distance_x_[m]'])
            value = float(content[i]['distance_x_[m]'])
            new_value = value / 1.33
            new_values.append(new_value)
            content[i]['distance_x_[m]'] = str(new_value)  # 新しい値で置き換え
            print("New value:", content[i]['distance_x_[m]'])

    # CSVファイルに書き込む
    with open(file, 'w', encoding='utf8', newline='') as f:
        csvwriter = csv.DictWriter(f, fieldnames=content[0].keys(), delimiter='\t')
        csvwriter.writeheader()
        csvwriter.writerows(content)

Original value: 1.2269246500933837
New value: 0.9224997369123186
Original value: 3.250699153714591
New value: 2.4441347020410458
Original value: 1.2860870815068501
New value: 0.9669827680502632
Original value: 2.400240090164184
New value: 1.8046917971159275
Original value: 3.362240000000001
New value: 2.5280000000000005
Original value: 2.281877323211367
New value: 1.7156972354972684
Original value: 3.362240000000001
New value: 2.5280000000000005
Original value: 1.2893577018612363
New value: 0.9694418810986739
Original value: 3.8741927695561236
New value: 2.9129268944031
Original value: 1.2405986918038867
New value: 0.9327809712811178
Original value: 3.246851112066409
New value: 2.4412414376439164
Original value: 2.2371784246036355
New value: 1.6820890410553648
Original value: 1.7962359904024754
New value: 1.3505533762424626
Original value: 2.2392816987027975
New value: 1.6836704501524793
Original value: 3.873785692113218
New value: 2.9126208211377578
Original value: 3.6024000000000007
